# 自定义层  
所有的自定义层必须也是基于nn.module

# 1、构造一个没有任何参数的自定义层

In [17]:
import torch
import torch.nn.functional as F
from torch import nn

In [18]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x-x.mean()
    

In [19]:
layer=CenteredLayer()
layer(torch.tensor([1,2,3],dtype=float))

tensor([-1.,  0.,  1.], dtype=torch.float64)

定义层和定义网络没有什么区别，层也是nn.Module的一个子类

In [20]:
#可以应用定义的子类
net=nn.Sequential(nn.Linear(4,8),CenteredLayer())

In [21]:
net(torch.tensor([1,2,3,4],dtype=torch.float)).mean()

tensor(-2.9802e-08, grad_fn=<MeanBackward0>)

# 2、构造一个带有参数的自定义层  
参数也必须是parameter实例
当用nn.Parameter定义后 会自动加上梯度  
在PyTorch中，通过 nn.Parameter 定义的参数默认会自动启用梯度计算（即 requires_grad=True）。  
这是因为它被设计为模型的可训练参数，需要参与反向传播优化。  
如果传入的 Tensor 显式设置了 requires_grad=False，例如：

t = torch.tensor([1.0], requires_grad=False)  
param = nn.Parameter(t)  
则 param.requires_grad 会继承原张量的 False，不计算梯度。

In [47]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_units,units))
        self.bias=nn.Parameter(torch.randn(units))
    def forward(self,x):
        x=torch.matmul(x,self.weight.data)+self.bias.data
        return F.relu(x)

In [52]:
l=MyLinear(4,8)
l(torch.randn(4,4))

tensor([[2.0785, 2.7675, 0.0000, 0.7874, 5.1672, 0.9029, 1.9653, 0.0000],
        [1.9364, 2.6063, 0.0000, 0.0000, 1.1541, 0.0000, 0.1236, 0.6410],
        [0.9405, 2.9317, 0.0000, 0.0000, 0.3395, 0.0000, 1.8060, 0.8136],
        [2.9483, 3.8355, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972]])

使用自定义的层构建模型